# Week 6 - Classification models  

## Part 4: Travel mode choice - Hierarchical models

This part is where we start to make things more interesting :-)

We will revisit the original real world problem of travel model choice (with 4 classes), but this time we shall consider a hierarchical model. 

More on that later, for now the same stuff from part 2: imports, loading data, preprocessing, train/test split, etc.

Import required libraries:

In [71]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import pystan
import pystan_utils
from sklearn.utils import shuffle

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

Load data:

In [2]:
pop1 = pd.read_csv("SpotifyAudioFeatures2017.csv")
pop2 = pd.read_csv("SpotifyAudioFeatures2018.csv")
pop3 = pd.read_csv("SpotifyAudioFeatures2019.csv")

df_pop = pd.concat([pop1, pop2,pop3])
df_pop["genre"] = [1]*len(df_pop)

met1 = pd.read_csv("SpotifyAudioFeatures2017metal.csv")
met2 = pd.read_csv("SpotifyAudioFeatures2018metal.csv")
met3 = pd.read_csv("SpotifyAudioFeatures2019metal.csv")

df_met = pd.concat([met1, met2, met3])
df_met["genre"] = [2]*len(df_met)

df_clas = pd.read_csv("SpotifyAudioFeaturesclassical.csv")
df_clas["genre"] = [3]*len(df_clas)

rap1 = pd.read_csv("SpotifyAudioFeatures2017rap.csv")
rap2 = pd.read_csv("SpotifyAudioFeatures2018rap.csv")
rap3 = pd.read_csv("SpotifyAudioFeatures2019rap.csv")

df_rap = pd.concat([rap1, rap2, rap3])
df_rap["genre"] = [3]*len(df_rap)

df = pd.concat([df_pop,df_met,df_clas,df_rap])
#df.to_csv("genredata.csv")

In [73]:
# load csv
df_tracks = pd.read_csv("genredata.csv")
df_tracks = df_tracks.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
df_tracks = df_tracks.dropna(axis = 0)
df_tracks.head()

,Unnamed: 0,Unnamed: 0.1,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,genre
0,0,0,SUNMI,Gashina,0jFHMDRXxKaREor3hBEEST,74,0.055600,0.722,180000,0.833,0.000000,6,0.1950,-3.915,1,0.0940,93.941,4,0.530,1
1,1,1,Nelson Can,Break Down Your Walls,3r6nuiebZxOtdd9tKhZ5SV,40,0.632000,0.603,230060,0.551,0.000095,2,0.1110,-8.590,1,0.0398,107.968,4,0.226,1
2,2,2,199X,Gillette,5PVPkjFKLElXpshtXtXIFc,58,0.006350,0.608,120000,0.932,0.000783,7,0.3320,-4.077,1,0.0855,120.072,4,0.286,1
3,3,3,Yoke Lore,Truly Madly Deeply - Recorded at Spotify Studi...,0hLObGB9xRjuRVasHehmLI,68,0.826000,0.675,190393,0.368,0.001600,0,0.1360,-14.072,0,0.0374,106.001,4,0.176,1
4,4,4,Martin Garrix,Byte,20hsdn8oITBsuWNLhzr5eh,63,0.000636,0.597,285089,0.931,0.821000,7,0.0593,-4.026,0,0.0329,128.031,4,0.267,1


Preprocess data:

In [76]:
grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()
df = df_tracks
df = shuffle(df)
df.shape

In [79]:
# separate between features/inputs (X) and target/output variables (y)
mat = df.drop(['artist_name','track_name','track_id'],axis = 1)
mat = mat.values
X = mat.astype("float")
print(X.shape)
y = mat[:,-1].astype("int")
print(y.shape)
ind = mat[:,1].astype("int")
print(ind.shape)

(13200, 15)
(13200,)
(13200,)


In [80]:
# standardize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

Train/test split:

In [82]:
train_perc = 0.66 # percentage of training data
split_point = int(train_perc*len(y))
perm = np.random.permutation(len(y))
ix_train = perm[:split_point]
ix_test = perm[split_point:]
X_train = X[ix_train,:]
X_test = X[ix_test,:]
ind_train = ind[ix_train]
ind_test = ind[ix_test]
y_train = y[ix_train]
y_test = y[ix_test]
print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))

num train: 8712
num test: 4488


In [85]:
X_train

array([[-1.80856427,  1.13743298, -0.54915804, ...,  0.2814087 ,
        -0.04574581,  0.95365832],
       [ 1.04139254, -0.76984895,  1.00469329, ...,  0.2814087 ,
         0.63759007, -1.10999575],
       [ 0.50221152, -1.23803129,  0.2813487 , ...,  0.2814087 ,
         0.59691531, -1.10999575],
       ...,
       [ 1.07990547, -0.62218841,  0.75911334, ...,  0.2814087 ,
         0.18203281, -1.10999575],
       [ 0.77180203, -0.59265631,  0.98683293, ...,  0.2814087 ,
         0.60098279, -1.10999575],
       [-1.65451255,  1.10051785, -0.78134272, ...,  0.2814087 ,
        -1.05447973,  0.95365832]])

Our baseline logistic regression model from sklearn:

In [87]:
# create and fit logistic regression model
logreg = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
logreg.fit(X_train, y_train)

# make predictions for test set
y_hat = logreg.predict(X_test)
print("Predictions:", y_hat)
print("True values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

Predictions: [3 3 1 ... 1 3 1]
True values: [3 3 1 ... 1 3 1]
Accuracy: 1.0


## Hierarchical logistic regression in STAN

We will now implement a hierarchical logistic regression. The motivation is actually quite simple. Our dataset consists of multiple observations from various individuals. However, when we build our original logistic regression in STAN, our specification assumes that all individuals share a unique set of bias (alpha) coefficients (beta). In other words, this is equivalent to assuming, for example, that all individuals are equally biased towards a given mode (e.g. car). This is obviously a very strong assumption, right? We should allow different individuals to have different biases (alpha). (We could also consider different coefficients per individual, but for the sake of simplicy, we will just focus on the bias parameters)

This can be done by placing a hierarchical prior on the intercepts (alpha). The generative process then becomes:

1. For each class $c \in \{1,\dots,C\}$
    2. Draw global intercept mean $\mu_c \sim \mathcal{N}(0,10)$
    3. Draw global intercept variance $\sigma_c \sim \mbox{Cauchy}(0,10)$
    5. Draw coefficients $\boldsymbol\beta_c \sim \mathcal{N}(\textbf{0},10 \, \textbf{I})$ (this the same as before...)
    6. For each individual $i \in \{1,\dots,I\}$
        4. Draw $\alpha_{i,c}$ such that $\alpha_{i,c} \sim \mathcal{N}(\mu_c,\sigma_c)$

6. For each data point $n=\{1,\dots,N\}$
    7. Draw target class $y_n \sim \mbox{Multinomial}(\mbox{Softmax}(\textbf{x}_n,\boldsymbol\alpha_{i_n},\boldsymbol\beta_1,\dots,\boldsymbol\beta_C))$
    
where $i_n$ is the individual identifier for person $n$, and $\boldsymbol\mu=\{\mu_1\dots\mu_C\}$ and $\boldsymbol\sigma=\{\sigma_1\dots\sigma_C\}$.

Notice that now, instead of a single intercept per class $\alpha_c$ for all individual, we now have a vector of intercepts $\boldsymbol\alpha_c$ for each class $c$: one intercept parameter per individual! However, all these intercept share a global (population-level) prior.

Lets try to implement this in STAN. Can you do it? :-) 

In [88]:
# define Stan model
model_definition = """
data {
    int<lower=0> N;
    int<lower=1> D;
    int<lower=1> C;
    int<lower=1> I;
    int ind[N];
    matrix[N,D] X;
    int<lower=0,upper = C> y[N];
}
parameters {
    matrix[C,D] beta;
    matrix[I,C] alpha; // i individer og c classes, intercept/bias for hver class, 4 intercepts for hver
    
    vector[C] mu_prior;
    vector<lower=0>[C] sigma_prior;
    
}

model {
    for(c in 1:C){
        mu_prior[c] ~ normal(0,1);
        sigma_prior[c] ~ normal(0,1);
        beta[c] ~ normal(0,1);
        
        for(i in 1:I){
            alpha[i,c] ~ normal(mu_prior[c],sigma_prior[c]);
        }
    }
    for(n in 1:N){
        y[n] ~ categorical(softmax(alpha[ind[n],:]' + beta*X[n]'));
    }
}

"""

Prepare input data for STAN, compile STAN program and run inference using ADVI (much faster in this case):

In [89]:
# prepare data for Stan model
N, D = X_train.shape
C = int(y_train.max())
I = ind.max()
print("N = %d, D = %d, C = %d, I = %d" % (N,D,C,I))
data = {'N': N, 'D': D, 'C': C, 'I':I, 'ind':ind_train, 'X': X_train, 'y': y_train}

N = 8712, D = 15, C = 3, I = 0


In [90]:
%%time
# create Stan model object
sm = pystan.StanModel(model_code=model_definition)
fit = sm.vb(data=data, iter=10000, algorithm="meanfield", grad_samples=10, seed=42, verbose=True)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_47fb036e992c954f28293ae82a359042 NOW.
C:\Users\clara\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\clara\AppData\Local\Temp\tmpzoe38xa7\stanfit4anon_model_47fb036e992c954f28293ae82a359042_7670231639174555707.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
C:\Users\clara\Anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


ValueError: Exception: anon_model_47fb036e992c954f28293ae82a359042_namespace::anon_model_47fb036e992c954f28293ae82a359042: I is 0, but must be greater than or equal to 1  (in 'unknown file name' at line 6)


Lets plot the posterior distributions of some of the parameters of our model (you may have called these variables something else...):

In [91]:
pystan_utils.vb_plot_variables(fit, "mu_prior") #mu_prior
# Hyper-priors for each class for all the 

NameError: name 'fit' is not defined

In [ ]:
pystan_utils.vb_plot_variables(fit, "sigma_prior")

We can now use the inferred posteriors to make predictions. Lets first use the "pystan_utils" package to extract the expected values of the posterior distribution of the model parameters:

In [ ]:
# get fitted parameters
mu_prior = pystan_utils.vb_extract_variable(fit, "mu_prior", var_type="vector")
sigma_prior = pystan_utils.vb_extract_variable(fit, "sigma_prior", var_type="vector")
alpha = pystan_utils.vb_extract_variable(fit, "alpha", var_type="matrix", dims=(C,I))
beta = pystan_utils.vb_extract_variable(fit, "beta", var_type="matrix", dims=(C,D))

Using expected values of the parameters, we can make predictions for the testset. However, we need to account for the fact that we now have different bias parameters per-individual, and adapt the code for making predictions accordingly. Make sure that you understand the code below. As always, if something is not 100% clear, ask! :-)

In [ ]:
# make predictions for test set
y_hat = alpha[:,ind_test-1] + np.dot(beta, X_test.T)
y_hat = np.argmax(y_hat, axis=0) + 1
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

Now, that is a signficant improvement, right? We improved the accuracy of our model from 67.9% to about 78.4%! (Hopefully you were able to obtain a similar or even better result :-)

Did you see how your prior knowledge about the problem can make a substantial difference when building a model for it? This is how things are done in the model-based machine learning approach!

Given the posterior distributions inferred by STAN, we can even analyse the biases of different individuals identified by our model:

In [ ]:
for i in range(I):
    print(i, alpha[:,i])

Perhaps a histogram allows for a better global analysis:

In [ ]:
# histogram of biases towards mode 4 (car)
plt.hist(alpha[3,:])
plt.title("Biases towards mode 4")
plt.xlabel("alpha[4]")
plt.show()

We can observe that, for most individuals the biases is around 0. However, we can also see that a few individuals really love their cars!

Reflection exercise: can you think of ways in which you could use this model to try to identify policies (e.g. price changes or making terminals more efficient) that would allow to shift people's travel mode choices away from the car (e.g. towards public transport)?